In [28]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [29]:
import ibgeparser
# import da classe principal
from ibgeparser.microdados import Microdados
# import dos enums para facilitar as buscas
from ibgeparser.enums import Anos, Estados, Modalidades


In [30]:
# procura todos os nomes de arquivos da pasta "microdados-ibge" e coloca numa lista
data_path = Path("microdados-ibge")
file_list = list(data_path.glob("Amostra*.csv"))

colunas = {
    "V6036" : "Idade",
    "V6400" : "NivelInstrucao",
    "V6352" : "CursoGraduacaoCodigo",
    "V6354" : "CursoMestradoCodigo",
    "V6356" : "CursoDoutoradoCodigo",
    "V6461" : "OcupacaoCodigo",
    "V6471" : "AtividadeCodigo",
    "V6462" : "CBO-Domiciliar",
    "V6472" : "CNAE-Domiciliar",
    "V6511" : "ValorRenda",
    "V6514" : "QuantidadeSM",
    "V0601" : "Genero",
    "V0656" : "RendimentoAposentadoriaPensao"}

# Definir tipos de dados para cada coluna
tipos_float = ["V6511", "V0656"]  # ValorRenda e RendimentoAposentadoriaPensao
tipos_int = [col for col in colunas.keys() if col not in tipos_float]

dtype_dict = {col: 'float64' if col in tipos_float else 'Int64' for col in colunas.keys()}

print(f"Processando {len(file_list)} arquivos...")

# Percorra a lista de arquivos, extraia o estado do nome, abra o CSV, mantenha apenas as colunas definidas no dicionário "colunas", 
# e substitua o nome pelo valor do dicionário. adicione uma coluna extra chamada Estado e coloque o estado detectado nela.
# Consolide todos os dataframes em um único e grave no arquivo "rodolfo/original.csv"

# Processar arquivo por arquivo e acumular em lista para gravar em Parquet
output_file = "rodolfo/original_filtrado.parquet"
df_list = []
total_filtrados = 0

for i, file_path in enumerate(file_list, 1):
    file_name = file_path.name
    if "SP1" in file_name or "SP2" in file_name:
        estado = "SP"
    else:
        estado = file_name.split(".csv")[0][-2:]
    
    print(f"[{i}/{len(file_list)}] {file_name} -> {estado}...", end=" ")
    
    # Ler apenas as colunas necessárias com os tipos corretos
    df = pd.read_csv(file_path, sep=",", usecols=list(colunas.keys()), dtype=dtype_dict, low_memory=False)
    
    # Filtrar apenas registros com NivelInstrucao == 4
    registros_antes = len(df)
    df = df[df["V6400"] == 4]
    registros_depois = len(df)
    
    df = df.rename(columns=colunas)
    df["Estado"] = estado
    
    # Adicionar à lista
    df_list.append(df)
    
    print(f"{registros_depois:,} registros (de {registros_antes:,})")
    total_filtrados += registros_depois

# Consolidar e gravar em Parquet (preserva tipos automaticamente)
print("\nConsolidando dados...")
df_consolidado = pd.concat(df_list, ignore_index=True)
df_consolidado.to_parquet(output_file, compression='snappy', index=False)

print(f"\nArquivo consolidado gravado em: {output_file}")
print(f"Total de registros com NivelInstrucao=4: {total_filtrados:,}")
print(f"Tamanho do arquivo: {Path(output_file).stat().st_size / (1024**2):.2f} MB")



# Liberar memóriadel df_list, df_consolidado

Processando 28 arquivos...
[1/28] Amostra_Pessoas_23_CE.csv -> CE... 26,782 registros (de 846,163)
[2/28] Amostra_Pessoas_14_RR.csv -> RR... 26,782 registros (de 846,163)
[2/28] Amostra_Pessoas_14_RR.csv -> RR... 2,742 registros (de 63,764)
[3/28] Amostra_Pessoas_43_RS.csv -> RS... 2,742 registros (de 63,764)
[3/28] Amostra_Pessoas_43_RS.csv -> RS... 83,113 registros (de 1,388,442)
[4/28] Amostra_Pessoas_22_PI.csv -> PI... 83,113 registros (de 1,388,442)
[4/28] Amostra_Pessoas_22_PI.csv -> PI... 14,176 registros (de 496,476)
[5/28] Amostra_Pessoas_11_RO.csv -> RO... 14,176 registros (de 496,476)
[5/28] Amostra_Pessoas_11_RO.csv -> RO... 8,225 registros (de 195,606)
[6/28] Amostra_Pessoas_12_AC.csv -> AC... 8,225 registros (de 195,606)
[6/28] Amostra_Pessoas_12_AC.csv -> AC... 3,639 registros (de 93,674)
[7/28] Amostra_Pessoas_25_PB.csv -> PB... 3,639 registros (de 93,674)
[7/28] Amostra_Pessoas_25_PB.csv -> PB... 17,770 registros (de 571,630)
[8/28] Amostra_Pessoas_17_TO.csv -> TO... 1

In [31]:
# Função para ler o arquivo consolidado com os tipos corretos
def ler_dados_consolidados(arquivo="rodolfo/original_limpo.parquet"):
    """
    Lê o arquivo consolidado de microdados do IBGE.
    
    Parâmetros:
    -----------
    arquivo : str
        Caminho do arquivo Parquet a ser lido (padrão: "rodolfo/original_limpo.parquet")
    
    Retorna:
    --------
    DataFrame com as colunas nos tipos corretos (preservados automaticamente pelo Parquet):
    - Inteiros: Idade, NivelInstrucao, CursoGraduacaoCodigo, CursoMestradoCodigo, 
                CursoDoutoradoCodigo, OcupacaoCodigo, AtividadeCodigo, CBO-Domiciliar,
                CNAE-Domiciliar, QuantidadeSM, Genero
    - Float: ValorRenda, RendimentoAposentadoriaPensao
    - String: Estado
    
    Vantagens do Parquet:
    - Leitura 5-10x mais rápida que CSV
    - Tipos de dados preservados automaticamente
    - Compressão nativa (~50-70% menor que CSV)
    - Suporte a leitura parcial de colunas
    """
    df = pd.read_parquet(arquivo)
    return df

# Exemplo de uso:
# df = ler_dados_consolidados()
# print(df.info())
# print(df.head())

# Ler apenas colunas específicas (feature do Parquet):
# df_parcial = pd.read_parquet("rodolfo/original_limpo.parquet", columns=['Idade', 'ValorRenda', 'Estado'])

In [32]:
df = ler_dados_consolidados('rodolfo/original_filtrado.parquet')
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126884 entries, 0 to 1126883
Data columns (total 14 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   Genero                         1126884 non-null  Int64  
 1   Idade                          1126884 non-null  Int64  
 2   NivelInstrucao                 1126884 non-null  Int64  
 3   CursoGraduacaoCodigo           1072460 non-null  Int64  
 4   CursoMestradoCodigo            39492 non-null    Int64  
 5   CursoDoutoradoCodigo           14932 non-null    Int64  
 6   OcupacaoCodigo                 925821 non-null   Int64  
 7   AtividadeCodigo                925821 non-null   Int64  
 8   ValorRenda                     918556 non-null   float64
 9   QuantidadeSM                   925821 non-null   Int64  
 10  RendimentoAposentadoriaPensao  1126884 non-null  float64
 11  CBO-Domiciliar                 925821 non-null   Int64  
 12  CNAE-Domicilia

In [33]:
# ============================================
# ANÁLISE DE QUALIDADE DOS DADOS
# ============================================

print("\n" + "="*80)
print("ANÁLISE DE QUALIDADE DOS DADOS")
print("="*80)

# 1. Verificar valores nulos/ausentes
print("\n[1] VALORES AUSENTES POR COLUNA:")
print("-" * 50)
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
  'Coluna': missing_data.index,
  'Valores Ausentes': missing_data.values,
  'Percentual (%)': missing_percent.values
})
missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)
print(missing_df.to_string(index=False))

# Alertas críticos de dados ausentes
print("\n⚠️  ALERTAS DE DADOS AUSENTES:")
if missing_df['Percentual (%)'].max() > 50:
  print(f"   CRÍTICO: Colunas com mais de 50% de dados ausentes detectadas!")
if (df['ValorRenda'].isnull().sum() / len(df) * 100) > 15:
  print(f"   ATENÇÃO: ValorRenda tem {(df['ValorRenda'].isnull().sum() / len(df) * 100):.2f}% de valores ausentes")

# 2. Verificar valores duplicados
print("\n[2] VERIFICAÇÃO DE DUPLICATAS:")
print("-" * 50)
duplicates = df.duplicated().sum()
print(f"Registros duplicados completos: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")

# 3. Análise de valores inconsistentes
print("\n[3] ANÁLISE DE VALORES INCONSISTENTES:")
print("-" * 50)

# Idade
idade_invalida = df[(df['Idade'] < 18) | (df['Idade'] > 100)]
print(f"Idades fora do intervalo esperado (18-100): {len(idade_invalida):,}")
if len(idade_invalida) > 0:
  print(f"   ⚠️  Idades mínima: {df['Idade'].min()}, máxima: {df['Idade'].max()}")

# Gênero
generos_unicos = df['Genero'].unique()
print(f"Valores únicos de Gênero: {sorted([x for x in generos_unicos if pd.notna(x)])}")
if len(generos_unicos) > 2:
  print(f"   ⚠️  ATENÇÃO: Mais de 2 valores únicos detectados em Gênero!")

# ValorRenda
renda_negativa = df[df['ValorRenda'] < 0]
print(f"Valores de renda negativos: {len(renda_negativa):,}")
if len(renda_negativa) > 0:
  print(f"   ⚠️  ALERTA: Valores de renda negativos detectados!")

renda_outliers = df[df['ValorRenda'] > 50000]
print(f"Valores de renda > R$ 50.000: {len(renda_outliers):,} ({len(renda_outliers)/len(df[df['ValorRenda'].notna()])*100:.2f}%)")

# 4. Consistência entre variáveis relacionadas
print("\n[4] CONSISTÊNCIA ENTRE VARIÁVEIS:")
print("-" * 50)

# Pessoas com ocupação devem ter renda
com_ocupacao_sem_renda = df[(df['OcupacaoCodigo'].notna()) & (df['ValorRenda'].isna())]
print(f"Registros com Ocupação mas sem ValorRenda: {len(com_ocupacao_sem_renda):,}")
if len(com_ocupacao_sem_renda) > 0:
  print(f"   ⚠️  {len(com_ocupacao_sem_renda)/len(df)*100:.2f}% dos registros têm ocupação mas não têm renda registrada")

# CBO e CNAE devem estar juntos
cbo_sem_cnae = df[(df['CBO-Domiciliar'].notna()) & (df['CNAE-Domiciliar'].isna())]
cnae_sem_cbo = df[(df['CNAE-Domiciliar'].notna()) & (df['CBO-Domiciliar'].isna())]
print(f"Registros com CBO mas sem CNAE: {len(cbo_sem_cnae):,}")
print(f"Registros com CNAE mas sem CBO: {len(cnae_sem_cbo):,}")

# 5. Distribuição por Estado
print("\n[5] DISTRIBUIÇÃO POR ESTADO:")
print("-" * 50)
dist_estados = df['Estado'].value_counts().sort_index()
print(dist_estados)
print(f"\nEstados representados: {df['Estado'].nunique()}")

# 6. Estatísticas de renda
print("\n[6] ESTATÍSTICAS DE RENDA:")
print("-" * 50)
print(f"Renda média: R$ {df['ValorRenda'].mean():.2f}")
print(f"Renda mediana: R$ {df['ValorRenda'].median():.2f}")
print(f"Renda mínima: R$ {df['ValorRenda'].min():.2f}")
print(f"Renda máxima: R$ {df['ValorRenda'].max():.2f}")
print(f"Desvio padrão: R$ {df['ValorRenda'].std():.2f}")

# 7. Resumo de cursos
print("\n[7] ANÁLISE DE FORMAÇÃO ACADÊMICA:")
print("-" * 50)
print(f"Registros com Graduação: {df['CursoGraduacaoCodigo'].notna().sum():,} ({df['CursoGraduacaoCodigo'].notna().sum()/len(df)*100:.2f}%)")
print(f"Registros com Mestrado: {df['CursoMestradoCodigo'].notna().sum():,} ({df['CursoMestradoCodigo'].notna().sum()/len(df)*100:.2f}%)")
print(f"Registros com Doutorado: {df['CursoDoutoradoCodigo'].notna().sum():,} ({df['CursoDoutoradoCodigo'].notna().sum()/len(df)*100:.2f}%)")

# Pessoas com mestrado/doutorado mas sem graduação
mestrado_sem_grad = df[(df['CursoMestradoCodigo'].notna()) & (df['CursoGraduacaoCodigo'].isna())]
doutorado_sem_grad = df[(df['CursoDoutoradoCodigo'].notna()) & (df['CursoGraduacaoCodigo'].isna())]
if len(mestrado_sem_grad) > 0 or len(doutorado_sem_grad) > 0:
  print(f"   ⚠️  Mestrado sem Graduação: {len(mestrado_sem_grad):,}")
  print(f"   ⚠️  Doutorado sem Graduação: {len(doutorado_sem_grad):,}")

# 8. Sugestões de limpeza
print("\n" + "="*80)
print("RECOMENDAÇÕES PARA O CIENTISTA DE DADOS")
print("="*80)
print("\n✓ Considere criar uma coluna 'RendaValida' com flag para rendas válidas")
print("✓ Avaliar imputação de valores ausentes em 'ValorRenda' baseado em ocupação/atividade")
print("✓ Considere criar categorias de faixa etária para análises")
print("✓ Avaliar remoção ou tratamento de outliers em ValorRenda")
print("✓ Criar feature 'TemPosGraduacao' (mestrado ou doutorado)")
print("✓ Normalizar ou padronizar valores de renda para análises comparativas")
print("✓ Investigar registros com ocupação mas sem renda")
print("\n" + "="*80)


ANÁLISE DE QUALIDADE DOS DADOS

[1] VALORES AUSENTES POR COLUNA:
--------------------------------------------------
              Coluna  Valores Ausentes  Percentual (%)
CursoDoutoradoCodigo           1111952           98.67
 CursoMestradoCodigo           1087392           96.50
          ValorRenda            208328           18.49
      OcupacaoCodigo            201063           17.84
     AtividadeCodigo            201063           17.84
        QuantidadeSM            201063           17.84
      CBO-Domiciliar            201063           17.84
     CNAE-Domiciliar            201063           17.84
CursoGraduacaoCodigo             54424            4.83

⚠️  ALERTAS DE DADOS AUSENTES:
   CRÍTICO: Colunas com mais de 50% de dados ausentes detectadas!
   ATENÇÃO: ValorRenda tem 18.49% de valores ausentes

[2] VERIFICAÇÃO DE DUPLICATAS:
--------------------------------------------------
Registros duplicados completos: 188,247 (16.71%)

[3] ANÁLISE DE VALORES INCONSISTENTES:
---------

In [35]:
# Remover registros com idade maior que 100 anos ou menor que 18 anos
df_cleaned = df[(df['Idade'] <= 100) & (df['Idade'] >= 18)]
print(f"\nRegistros após remoção de idades inválidas: {len(df_cleaned):,} (removidos: {len(df) - len(df_cleaned):,})")

# Remover registros que não tenha curso de graduação.
df_cleaned = df_cleaned[df_cleaned['CursoGraduacaoCodigo'].notna()]
print(f"Registros após remoção de quem não tem graduação: {len(df_cleaned):,} (removidos: {len(df) - len(df_cleaned):,})")

# # Remover os dados que não tem renda registrada.
# df_cleaned = df_cleaned[df_cleaned['ValorRenda'].notna()]
# print(f"Registros após remoção de quem não tem renda registrada: {len(df_cleaned):,} (removidos: {len(df) - len(df_cleaned):,})")

df = df_cleaned

# Salvar em Parquet (mais eficiente que CSV)
df.to_parquet("rodolfo/original_limpo.parquet", compression='snappy', index=False)

print(f"\nArquivo limpo salvo em: rodolfo/original_limpo.parquet")
print(f"Tamanho: {Path('rodolfo/original_limpo.parquet').stat().st_size / (1024**2):.2f} MB")


Registros após remoção de idades inválidas: 1,126,430 (removidos: 454)
Registros após remoção de quem não tem graduação: 1,072,007 (removidos: 54,877)

Arquivo limpo salvo em: rodolfo/original_limpo.parquet
Tamanho: 9.08 MB

Arquivo limpo salvo em: rodolfo/original_limpo.parquet
Tamanho: 9.08 MB
